In [1]:
!pip install gradio pinecone sentence-transformers openai



In [6]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import openai
#from pydantic import BaseModel


PINECONE_API_KEY = "pcsk_2czr3A_58ustdWMomnjZ7p1P3P5UaNPagR9h61WNiW6iAeXFiDkH3gWrNoqpbxFghTZYCY"
INDEX_NAME = "bangla-book-index"
REGION = "us-east-1"
OPENAI_API_KEY = ""

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
model = SentenceTransformer('intfloat/multilingual-e5-large')
openai.api_key = ""
#class QueryRequest(BaseModel):
    #question: str

def rag_qa(user_query):

    # 1. Embed the query
    query_emb = model.encode(f"query: {user_query}")

    # 2. Retrieve top-K chunks
    K = 5
    results = index.query(
        vector=query_emb.tolist(),
        top_k=K,
        include_metadata=True
    )
    retrieved_chunks = [match['metadata']['text'] for match in results['matches']]
    context = "\n\n".join([f"[Passage {i+1}]\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])

    # 3. Build LLM prompt (copy from your notebook)
    llm_prompt = f"""
You are a Bangla QA assistant for textbooks. The question can be asked in both english or bangla but you need to answer in bangla.it in Bangla.

You have been provided with several retrieved passages from a textbook. These passages may include:
- Explanatory or narrative text (main passages)
- Tables
- MCQ (multiple choice questions) and their options or answer keys
- Lists and other formatted content

**Instructions:**
1. **If the question includes the word 'বাগধারাটি', first look for an answer in the word meaning section (such as sections titled 'বাগধারা', 'বাগধারার অর্থ', or where word meanings are explained).**
2. **First, carefully read the main explanatory/narrative passages. Try to find the answer to the user's question in these main text sections.**
3. **If you do not find the answer in the passages, then check the MCQ, option lists, tables, or answer keys in the retrieved text.**
4. If you still cannot find the answer, reply only with: 'Not found in the book.' Never make up information or use outside knowledge.

If multiple sources in the passages or MCQ part provide relevant details, synthesize the answer based on all of them.


Below are some examples:

User Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
Expected Answer: শুম্ভুনাথ

User Question: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Expected Answer: মামাকে

User Question: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Expected Answer: ১৫ বছর

Here are the most relevant passages:
{context}

Answer the following question using ONLY the information in the passages above.

Question: {user_query}
"""

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are an expert educational QA assistant."},
            {"role": "user", "content": llm_prompt}
        ],
        temperature=0.2,
        max_tokens=512
    )
    answer = response.choices[0].message.content
    return answer.strip()

# Create Gradio interface
iface = gr.Interface(
    fn=rag_qa,
    inputs=gr.Textbox(label="Ask a Question (Bangla or English)"),
    outputs=gr.Textbox(label="Model Answer"),
    title="Bangla Textbook RAG QA Bot",
    description="Ask any Bangla/English question from the textbook. The model will answer only from book content."
)

iface.launch(share=True)  # use share=True for a public link in Colab


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c538186c98add9fb16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
